# 📊 JPMorgan European Equity Thesis – Backtest Analysis

This notebook performs **end-to-end backtest analysis** for the JPMorgan European Equity Thesis strategy.

It is designed to work with your project structure:

- `src/analytics/backtest/engine.py`
- `src/analytics/risk/var_calculator.py` (or RiskAnalytics helper)
- `src/utils/math_utils.py`
- `src/data/connectors/yahoo.py`

You can use this notebook to:

1. Load and validate historical price data (STOXX vs S&P 500)
2. Run your **JPM thesis strategy backtest** vs **buy-and-hold benchmark**
3. Analyze **returns, drawdowns, risk, Sharpe, Sortino, VaR/CVaR**
4. Run **scenario and sensitivity analysis**
5. Export key results for your dashboard & reports


## 1️⃣ Environment & Imports

This section sets up paths and imports the project modules. Make sure you run this notebook from the **project root** (where `src/` lives).

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# Ensure project root is in path
PROJECT_ROOT = Path.cwd()
if (PROJECT_ROOT / 'src').exists():
    sys.path.append(str(PROJECT_ROOT))
else:
    # If running from notebooks/ directory, go one level up
    PROJECT_ROOT = PROJECT_ROOT.parent
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

# Project imports (match your structure)
from src.analytics.backtest.engine import BacktestEngine
from src.utils.math_utils import (
    annualize_return,
    annualize_volatility,
    sharpe_ratio,
    sortino_ratio,
    max_drawdown,
    value_at_risk,
    expected_shortfall,
)
from src.data.connectors.yahoo import YahooMarketData

plt.style.use('seaborn-v0_8')


## 2️⃣ Load Historical Data (STOXX vs S&P 500)

We fetch price history for:
- **`^STOXX50E`** – Euro STOXX 50 (Europe proxy)
- **`^GSPC`** – S&P 500 (US benchmark)

You can change tickers to STOXX 600 equivalents or ETFs if required.

In [ ]:
yahoo = YahooMarketData()

start_date = "2020-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

tickers = ["^STOXX50E", "^GSPC"]

prices_df = yahoo.get_history_bulk(tickers=tickers, start=start_date, end=end_date)
prices_df = prices_df.dropna(how="all")

prices_df.head()

### 2.1 Basic Checks & Visualization

We quickly visualize both indices and check for missing data.

In [ ]:
# Quick sanity checks
print("Data shape:", prices_df.shape)
print("Start date:", prices_df.index.min())
print("End date:", prices_df.index.max())
print("Missing values per column:\n", prices_df.isna().sum())

fig, ax = plt.subplots(figsize=(10, 5))
prices_df["^STOXX50E"].plot(ax=ax, label="Euro STOXX 50")
prices_df["^GSPC"].plot(ax=ax, label="S&P 500")
ax.set_title("Index Levels – Euro STOXX 50 vs S&P 500")
ax.set_ylabel("Index Level")
ax.legend()
plt.show()

## 3️⃣ Run Backtest via `BacktestEngine`

We use your **strategy engine** from `src/analytics/backtest/engine.py`.

The strategy logic (inside your engine) typically does:

- Compute **relative performance** (Europe vs US)
- Define regimes based on thresholds
  - Oversold Europe → **overweight Europe**
  - Neutral → partial exposure
  - Overextended Europe → reduce exposure
- Rebalance portfolio based on signals


In [ ]:
# Initialize backtest engine
engine = BacktestEngine(start_date=start_date, end_date=end_date)

# If your BacktestEngine accepts external price data, you can wire it like this:
try:
    results = engine.run_backtest(price_data=prices_df)
except TypeError:
    # Fallback: assume engine fetches its own data internally
    results = engine.run_backtest()

results.keys()

### 3.1 Inspect Equity Curves & Metrics

We expect `results` to contain keys like:

- `dates`
- `strategy_equity`
- `buyhold_equity`
- `strategy_metrics`
- `buyhold_metrics`


In [ ]:
dates = pd.to_datetime(results["dates"])
strategy_eq = pd.Series(results["strategy_equity"], index=dates, name="Strategy")
buyhold_eq = pd.Series(results["buyhold_equity"], index=dates, name="Buy & Hold")

strategy_metrics = results.get("strategy_metrics", {})
buyhold_metrics = results.get("buyhold_metrics", {})

display(pd.DataFrame({"Strategy": strategy_metrics, "Buy & Hold": buyhold_metrics}))

### 3.2 Plot Equity Curves (Plotly)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=strategy_eq.index,
    y=strategy_eq.values,
    mode="lines",
    name="JPM Thesis Strategy",
    line=dict(width=3)
))
fig.add_trace(go.Scatter(
    x=buyhold_eq.index,
    y=buyhold_eq.values,
    mode="lines",
    name="STOXX Buy & Hold",
    line=dict(width=2, dash="dash")
))
fig.update_layout(
    title="Cumulative Equity Curve – Strategy vs Buy & Hold",
    xaxis_title="Date",
    yaxis_title="Cumulative Return (Indexed)",
    template="plotly_white",
    hovermode="x unified",
)
fig.show()

## 4️⃣ Risk & Performance Metrics (from `math_utils`)

We recompute some core metrics using `src/utils/math_utils.py`.

In [ ]:
# Compute returns from equity curves
strategy_ret = strategy_eq.pct_change().dropna()
buyhold_ret = buyhold_eq.pct_change().dropna()

metrics_custom = {
    "Strategy": {
        "Ann. Return": annualize_return(strategy_ret),
        "Ann. Vol": annualize_volatility(strategy_ret),
        "Sharpe": sharpe_ratio(strategy_ret, risk_free_rate=0.02),
        "Sortino": sortino_ratio(strategy_ret, risk_free_rate=0.02),
        "Max Drawdown": max_drawdown(strategy_eq)[0],
        "VaR 95": value_at_risk(strategy_ret, 0.95),
        "ES 95": expected_shortfall(strategy_ret, 0.95),
    },
    "Buy & Hold": {
        "Ann. Return": annualize_return(buyhold_ret),
        "Ann. Vol": annualize_volatility(buyhold_ret),
        "Sharpe": sharpe_ratio(buyhold_ret, risk_free_rate=0.02),
        "Sortino": sortino_ratio(buyhold_ret, risk_free_rate=0.02),
        "Max Drawdown": max_drawdown(buyhold_eq)[0],
        "VaR 95": value_at_risk(buyhold_ret, 0.95),
        "ES 95": expected_shortfall(buyhold_ret, 0.95),
    },
}

pd.DataFrame(metrics_custom).style.format({
    ("Strategy", "Ann. Return"): "{:.2%}",
    ("Strategy", "Ann. Vol"): "{:.2%}",
    ("Strategy", "Max Drawdown"): "{:.2%}",
    ("Strategy", "VaR 95"): "{:.2%}",
    ("Strategy", "ES 95"): "{:.2%}",
    ("Buy & Hold", "Ann. Return"): "{:.2%}",
    ("Buy & Hold", "Ann. Vol"): "{:.2%}",
    ("Buy & Hold", "Max Drawdown"): "{:.2%}",
    ("Buy & Hold", "VaR 95"): "{:.2%}",
    ("Buy & Hold", "ES 95"): "{:.2%}",
})

## 5️⃣ Drawdown Analysis

Visualizing drawdowns over time is a critical part of JPMorgan-style risk review.

In [ ]:
# Compute drawdowns
from src.utils.math_utils import drawdown_series

dd_strategy = drawdown_series(strategy_eq)
dd_buyhold = drawdown_series(buyhold_eq)

fig, ax = plt.subplots(figsize=(10, 4))
dd_strategy.plot(ax=ax, label="Strategy")
dd_buyhold.plot(ax=ax, label="Buy & Hold", alpha=0.7)
ax.set_title("Drawdown Over Time")
ax.set_ylabel("Drawdown")
ax.legend()
plt.show()

## 6️⃣ Scenario / Sensitivity Analysis

We can optionally shock parameters (e.g., thresholds, transaction costs) and rerun multiple backtests to see impact.

In [ ]:
# Example: vary a parameter (e.g., relative performance threshold)

scenarios = [
    {"name": "Base", "rel_threshold": -0.10},
    {"name": "More Aggressive", "rel_threshold": -0.15},
    {"name": "More Conservative", "rel_threshold": -0.05},
]

scenario_results = []

for sc in scenarios:
    print(f"Running scenario: {sc['name']}")
    try:
        res = engine.run_backtest(price_data=prices_df, rel_threshold=sc["rel_threshold"])
    except TypeError:
        # If engine doesn't support parameter, skip
        continue
    strat_ret = pd.Series(res["strategy_equity"], index=pd.to_datetime(res["dates"])).pct_change().dropna()
    scenario_results.append({
        "Scenario": sc["name"],
        "Ann. Return": annualize_return(strat_ret),
        "Ann. Vol": annualize_volatility(strat_ret),
        "Sharpe": sharpe_ratio(strat_ret, risk_free_rate=0.02),
    })

pd.DataFrame(scenario_results).style.format({
    "Ann. Return": "{:.2%}",
    "Ann. Vol": "{:.2%}",
})

## 7️⃣ Export Results for Dashboard / Reports

We can export cleaned backtest outputs to CSV/JSON for use by:
- `app.py` (Streamlit dashboard)
- `src/reporting/pdf_generator.py`
- `src/reporting/excel_exporter.py`

In [ ]:
EXPORT_DIR = PROJECT_ROOT / "data" / "exports"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

backtest_df = pd.DataFrame({
    "date": strategy_eq.index,
    "strategy_equity": strategy_eq.values,
    "buyhold_equity": buyhold_eq.values,
})

csv_path = EXPORT_DIR / "backtest_equity_curve.csv"
json_path = EXPORT_DIR / "backtest_results.json"

backtest_df.to_csv(csv_path, index=False)
pd.DataFrame({"strategy_metrics": strategy_metrics, "buyhold_metrics": buyhold_metrics}).to_json(
    json_path, orient="index"
)

print("Saved:", csv_path)
print("Saved:", json_path)

## ✅ Summary

In this notebook, we:

1. Loaded historical data for **Euro STOXX 50 vs S&P 500**
2. Ran your **JPM European Thesis Strategy** via `BacktestEngine`
3. Compared against **buy-and-hold benchmark**
4. Computed risk/return metrics using `math_utils`
5. Visualized **equity curves** and **drawdowns**
6. Performed a simple **scenario analysis**
7. Exported results for the **dashboard** & **reports**

You can now plug these outputs directly into:
- `app.py` charts
- PDF/Excel reports
- Monitoring and alert thresholds
